In [1]:
import pandas as pd
import numpy as np
from nltk.corpus import stopwords
from sklearn.metrics.pairwise import linear_kernel
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.decomposition import LatentDirichletAllocation
from ast import literal_eval
import re
import random
import matplotlib.pyplot as plt

In [2]:
trip = pd.read_csv(r"C:\Users\Eswar\Desktop\Techknocks\cleartrip.csv")

In [3]:
trip.columns

Index(['address', 'area', 'city', 'cleartrip_seller_rating', 'country',
       'crawl_date', 'hotel_description', 'hotel_facilities',
       'hotel_star_rating', 'image_count', 'image_urls', 'landmark',
       'latitude', 'locality', 'longitude', 'pageurl', 'property_id',
       'property_name', 'property_type', 'province', 'qts', 'room_area',
       'room_count', 'room_facilities', 'room_type', 'similar_hotel',
       'sitename', 'state', 'tad_review_count', 'tad_review_rating',
       'tad_stay_review_rating', 'tripadvisor_seller_rating', 'uniq_id'],
      dtype='object')

In [4]:
C= trip['tad_review_rating'].mean()
C

3.7506979341150197

In [5]:
m= trip['tad_review_count'].quantile(0.9)
m

247.9000000000001

In [6]:
q_hotels = trip.copy().loc[trip['tad_review_count'] >= m]
q_hotels.shape

(359, 33)

In [7]:
def weighted_rating(trip, m=m, C=C):
    v = trip['tad_review_count']
    R = trip['tad_review_rating']
    # Calculation based on the IMDB formula
    return (v/(v+m) * R) + (m/(m+v) * C)

In [8]:
q_hotels['score'] = q_hotels.apply(weighted_rating, axis=1)

In [9]:
q_hotels = q_hotels.sort_values('score', ascending=False)
q_hotels.head(10)

,address,area,city,cleartrip_seller_rating,country,crawl_date,hotel_description,hotel_facilities,hotel_star_rating,image_count,...,room_type,similar_hotel,sitename,state,tad_review_count,tad_review_rating,tad_stay_review_rating,tripadvisor_seller_rating,uniq_id,score
4911,"Rambagh Circle, Opp Central Park, Bhawani Sing...",SMS Stadium-Rambagh Circle,Jaipur,NaN,INDIA,2016-09-01,"Armagh Palace Hotel, Jaipur lures business tra...",Basic Room Amenities:Air Conditioning|Wake-Up ...,5 Star hotel,27,...,Palace Rooms,159767|1324676|375354|332915|40611,https://www.cleartrip.com,Rajasthan,1368.0,5.0,Location::4.5|Rooms::5.0|Service::5.0|Value::4...,Top 1%,07945718ead5cf68712345e80b344944,4.808341
902,"Opp: St.Antony's Shrine, Federal Bank ATM Buil...",Fort Kochi,Kochi,NaN,INDIA,2016-08-16,"Saj Homestay, Kochi is a thoughtfully designed...",Basics:Internet|Air Conditioning|Doorman|House...,NaN,12,...,Standard AC,NaN,http://www.cleartrip.com,Kerala,1274.0,5.0,Location::4.5|Rooms::4.5|Service::5.0|Value::4...,Top 1%,771821b0ea88d49b8b3974ac99101cd5,4.796503
440,"1/328, Tower Road, Fort Cochin",Fort Kochi,Kochi,NaN,INDIA,2016-08-16,"Hotel Old Harbour, Kochi, is a perfect blend o...",Basics:Internet|Doorman|Housekeeping|Doctor on...,3 Star hotel,11,...,Superior Room,714308|1332834|720275|725723|331892,http://www.cleartrip.com,Kerala,1225.0,5.0,Location::5.0|Rooms::5.0|Service::5.0|Value::4...,Top 1%,b9cc888d284471ad06b8c357fea1b6e1,4.789733
4600,Goner Road,Jaipur Suburbs,Jaipur,NaN,INDIA,2016-09-01,The Oberoi Rajvilas boasts of comfortable room...,Basic Room Amenities:Air Conditioning|Wake-Up ...,5 Star hotel,20,...,Luxury Tent,159767|1324676|375354|332915|40587,https://www.cleartrip.com,Rajasthan,1109.0,5.0,Location::4.5|Rooms::5.0|Service::5.0|Value::4...,Top 1%,12771a6e9e48bd464d3f1a43dd585cdb,4.771758
4762,"On Fort, Kund Para, Jaisalmer",Inside Jaisalmer Fort,Jaisalmer,NaN,INDIA,2016-09-01,"Hotel Victoria, Jaisalmer, is a resplendent pa...",Basics:Internet|Adjoining Rooms|Doorman|Housek...,3 Star hotel,42,...,Standard Room,1324136|714826|378862|709231|315577,https://www.cleartrip.com,Rajasthan,633.0,5.0,Location::5.0|Rooms::4.5|Service::5.0|Value::4...,Top 3%,5c66011895827cf3102b944cac69b092,4.648425
3856,"On the Seaward side of the Baga bridge, Baga",Baga Beach-North Goa,Goa,NaN,INDIA,2016-08-16,Overlooking the salient shores of Baga River!A...,Food &amp; Beverage:Bar|Restaurant|Coffee Shop...,5 Star hotel,20,...,Courtyard Room,159612|41652|40441|388991|1360608,http://www.cleartrip.com,Goa,588.0,5.0,Location::4.5|Rooms::5.0|Service::5.0|Value::4...,Top 1%,39a5e3291470e491b86aa4535231383e,4.629499
2374,"Thotadhahalli Estate, Kaimara (P)",Keremakki Estate,Chikmaglur,NaN,INDIA,2016-08-16,"Thotadhahalli Homestay, Chikmagalur nestled in...",Food &amp; Beverage:Restaurant|Personal Servic...,NaN,8,...,Double Bed Rooms with Attached Bath,NaN,http://www.cleartrip.com,Karnataka,517.0,5.0,Location::5.0|Rooms::5.0|Service::5.0|Value::4...,Top 1%,5fe3f6e22cd61e32dc17e6816aba4e25,4.595108
4231,"Opp. Golden Petrol Pump, Peepliyon ki dhaani, ...",Amer Fort,Jaipur,NaN,INDIA,2016-09-01,Looking for economic and quality stays? Well O...,Basics:Internet|Air Conditioning|Non-Smoking R...,3 Star hotel,12,...,Standard Room AC,1328858|1327648|1343330|1328856|1327766,https://www.cleartrip.com,Rajasthan,10259.0,4.5,Location::0.0|Rooms::0.0|Service::0.0|Value::0...,Top 1%,520e02c2bfb483b3110df7f08109a126,4.482321
4934,Palace Road Rajasthan,Bani Park,Jaipur,5.0,INDIA,2016-09-01,The needs of guests are catered to with briskn...,Food &amp; Beverage:Bar|Restaurant|Coffee Shop...,5 Star hotel,27,...,Executive Club Special,159767|1324676|778640|375354|332915,https://www.cleartrip.com,Rajasthan,2661.0,4.5,Location::4.5|Rooms::4.5|Service::4.5|Value::4...,Top 3%,49d4c20694c6414f3ced39830e595386,4.436144
3467,Arrossim Beach,Majorda Beach-South Goa,Goa,NaN,INDIA,2016-08-16,"Park Hyatt Goa Resort and Spa, Goa is an amiab...",Basic Room Amenities:Air Conditioning|Food &am...,

In [10]:
trip.columns

Index(['address', 'area', 'city', 'cleartrip_seller_rating', 'country',
       'crawl_date', 'hotel_description', 'hotel_facilities',
       'hotel_star_rating', 'image_count', 'image_urls', 'landmark',
       'latitude', 'locality', 'longitude', 'pageurl', 'property_id',
       'property_name', 'property_type', 'province', 'qts', 'room_area',
       'room_count', 'room_facilities', 'room_type', 'similar_hotel',
       'sitename', 'state', 'tad_review_count', 'tad_review_rating',
       'tad_stay_review_rating', 'tripadvisor_seller_rating', 'uniq_id'],
      dtype='object')

In [11]:
trip['hotel_description'].head()

0    Hotel Jiwan Jyoti, Kalimpong is an ultimate st...
1    Retire to the beautiful rooms of OYO Premium K...
2    OYO Rooms Bera Road in Jaisalmer has a royal c...
3    Situated in quite part of the city yet close t...
4    Hotel Gorakh Haveli, Jaisalmer, with hospitali...
Name: hotel_description, dtype: object

In [12]:
tfidf = TfidfVectorizer(stop_words='english')

#Replace NaN with an empty string
trip['hotel_description'] = trip['hotel_description'].fillna('')

#Construct the required TF-IDF matrix by fitting and transforming the data
tfidf_matrix = tfidf.fit_transform(trip['hotel_description'])

#Output the shape of tfidf_matrix
tfidf_matrix.shape


(5000, 20035)

In [13]:
# Compute the cosine similarity matrix
cosine_sim = linear_kernel(tfidf_matrix, tfidf_matrix)

In [14]:
#Construct a reverse map of indices and movie titles
indices = pd.Series(trip.index, index=trip['property_name']).drop_duplicates()

In [15]:
def get_recommendations(title, cosine_sim=cosine_sim):
    # Get the index of the movie that matches the title
    idx = indices[title]

    # Get the pairwsie similarity scores of all movies with that movie
    sim_scores = list(enumerate(cosine_sim[idx]))

    # Sort the movies based on the similarity scores
    sim_scores = sorted(sim_scores, key=lambda x: x[1], reverse=True)

    # Get the scores of the 10 most similar movies
    sim_scores = sim_scores[1:11]

    # Get the movie indices
    hotel_indices = [i[0] for i in sim_scores]

    # Return the top 10 most similar movies
    return trip['property_name'].iloc[hotel_indices]

In [16]:
trip['province'].head()

0     Kalimpong
1    Jamshedpur
2     Jaisalmer
3      Kakinada
4     Jaisalmer
Name: province, dtype: object

In [17]:
get_recommendations('Hotel Jivan Jyoti')

2627                    Hotel Valentino
2924                       Hotel Zodiac
2328    Khush Alaya - A Sterling Resort
2261                      Hotel Pradhan
2836                         Hotel Yuma
2300                         R J Resort
2698                       Hotel Meghma
2874              Hotel Pleasure Palace
2075                 Hotel Royal Palace
2921                      Viramma Villa
Name: property_name, dtype: object

In [18]:
get_recommendations('OYO Rooms Bera Road')

4085                          Gorbandh Palace
3752                 OYO Rooms Deorali Bazaar
4418    OYO Rooms Near Gadisar Lake Jaisalmer
4432                       Hotel Gajju Palace
4470     OYO Rooms Near Golden Fort Jaisalmer
4256                       Hotel Royal Haveli
10              Vista Rooms at Jaisalmer Fort
4282                       Hotel Shahi Palace
4212                          Hotel Akashdeep
3107                            Hotel Berries
Name: property_name, dtype: object

In [19]:
trip.columns

Index(['address', 'area', 'city', 'cleartrip_seller_rating', 'country',
       'crawl_date', 'hotel_description', 'hotel_facilities',
       'hotel_star_rating', 'image_count', 'image_urls', 'landmark',
       'latitude', 'locality', 'longitude', 'pageurl', 'property_id',
       'property_name', 'property_type', 'province', 'qts', 'room_area',
       'room_count', 'room_facilities', 'room_type', 'similar_hotel',
       'sitename', 'state', 'tad_review_count', 'tad_review_rating',
       'tad_stay_review_rating', 'tripadvisor_seller_rating', 'uniq_id'],
      dtype='object')

In [21]:
import pytz as tz
import matplotlib.pyplot as plt

from datetime import datetime
from sklearn.cluster import KMeans

from sklearn.metrics import silhouette_score

In [26]:
trip['crawl_date']

0       2016-08-16
1       2016-08-16
2       2016-08-16
3       2016-08-16
4       2016-08-16
           ...    
4995    2016-09-01
4996    2016-09-01
4997    2016-09-01
4998    2016-09-01
4999    2016-09-01
Name: crawl_date, Length: 5000, dtype: object

In [24]:
trip_df = trip[['uniq_id', 'landmark',
       'latitude', 'locality', 'longitude', 'property_id', 'crawl_date']]

In [25]:
venues = trip[['property_id',
       'property_name', 'latitude', 'longitude']]

In [29]:
def parse_datetime(s):
    tzone = tz.timezone("America/New_York") #parse_datetime
    utc = datetime.strptime(s, '%Y-%m-%d')
    return tz.utc.localize(utc).astimezone(tzone)

In [30]:
trip_df['ts'] = trip_df['crawl_date'].apply(lambda x: parse_datetime(x))
trip_df = trip_df.drop('crawl_date',axis=1,errors='ignore')

C:\Users\Eswar\New folder\lib\site-packages\ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """Entry point for launching an IPython kernel.


In [32]:
trip_df['date']  = trip_df['ts'].astype(object).apply(lambda x : x.date())
trip_df['time']  = trip_df['ts'].astype(object).apply(lambda x : x.time())

In [33]:
trip_df['weekday']  = trip_df['date'].astype(object).apply(lambda x : x.weekday())
trip_df['day']   = trip_df['date'].astype(object).apply(lambda x : x.day)
trip_df['hour']   = trip_df['time'].astype(object).apply(lambda x : x.hour)

In [43]:
trip_df['date'].unique()

array([datetime.date(2016, 8, 15), datetime.date(2016, 8, 31)],
      dtype=object)